# Wikipedia Word Analyzer

Author:  Devin Patel  
Purpose: To scrape a selection wikipedia articles and perform sentence analysis on it using TextBlob.  

Tutorial(s) Followed:
- [Scraping Data from a Real Website | Web Scraping in Python](https://www.youtube.com/watch?v=8dTpNajxaH0&list=WL&index=9&t=4s)  

## Scraper

In [ ]:
# Imports
from bs4 import BeautifulSoup   # For HTML parsing
import requests                 # HTTP requests
import textblob                 # For sentence analysis

In [ ]:
# Request the page
url = 'https://en.wikipedia.org/wiki/Toilet_paper_orientation'

page = requests.get(url)

# Parse the page
soop = BeautifulSoup(page.text, 'html')
soop